# A

## Step 01. Installing CrewAI packages

In [1]:
%%capture
!pip install crewai

In [2]:
%%capture
!pip install 'crewai[tools]'

## Step 02. Importing CrewAI library (Agent, Task and Crew module)

In [29]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, DirectoryReadTool, FileReadTool
from crewai.tools import BaseTool

# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
from google.colab import userdata


os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('Open_AI')

In [30]:
groq_llm = LLM(model="groq/llama3-70b-8192", temperature=0)

#deepseek_llm = LLM(model="groq/deepseek-r1-distill-llama-70b", temperature=0)
gemini_llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0
)

## Sales Agent

In [31]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    llm= gemini_llm,
    allow_delegation=False,
    verbose=True
)

### CrewAI Tools

- **SerperDevTool** : Used for Google Search
- **DirectoryReadTool** : Open a directory
- **FileReadTool** : Read a file

https://docs.crewai.com/en/concepts/tools

In [32]:
# local directory 'instructions' contains markdown files
# markdown files (.md) having instructions on how to deal with company of different sizes

directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
serper_dev_tool = SerperDevTool()



lead_profiling = Task (
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[serper_dev_tool, directory_read_tool, file_read_tool],
    agent=sales_rep_agent
)

## Lead Sales Agent

In [33]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    llm= gemini_llm,
    allow_delegation=False,
    verbose=True
)

### Custom Tool (SentimentAnalysisTool)

- Create a custom tool using crewAI's BaseTool class

- Every Tool needs to have a `name` and a `description`.
- Tool logic written in `_run` function
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.

In [34]:
class SentimentAnalysisTool(BaseTool):
    name : str = "Sentiment Analysis Tool"
    description : str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, argument : str) -> str:
        return "positive"


sentiment_analysis_tool = SentimentAnalysisTool()



personalized_outreach_task = Task (
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"

    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[serper_dev_tool, sentiment_analysis_tool],
    agent=lead_sales_rep_agent
)

## Crew

In [37]:
crew = Crew(
    tasks=[lead_profiling, personalized_outreach_task],
    agents=[sales_rep_agent, lead_sales_rep_agent],
    memory=False,
    manager_llm=gemini_llm,
    verbose=False
)


inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that    │
│  recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile,   │
│  focusing on key decision-makers, recent business developments, and potential needs that align with our         │
│  offerings. This task is crucial for tailoring our engagement strategy effectively.                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: Okay, I need to create a comprehensive report on DeepLearningAI. I'll start by gathering information  │
│  about the company background, key personnel, recent milestones, and potential needs. I'll use the available    │
│  tools to search for this information and then compile it into a detailed report.                               │
│  First, I'll use the "Search the internet with Serper" tool to find general information about DeepLearningAI.   │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"DeepLearningAI company information\"}"                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'DeepLearningAI company information', 'type': 'search', 'num': 10, 'engine':        │
│  'google'}, 'organic': [{'title': 'About - DeepLearning.AI', 'link': 'https://www.deeplearning.ai/about/',      │
│  'snippet': 'DeepLearning.AI is an education technology company that is empowering the global workforce to      │
│  build an AI-powered future through world-class education.', 'position': 1}, {'title': 'Deeplearning.ai -       │
│  Crunchbase Company Profile & Funding', 'link': 'https://www.crunchbase.com/organization/deeplearning-ai',      │
│  'snippet': 'Deeplearning.ai is company that explores the frontier of AI. The company is founded by Andrew Ng,  │
│  the former Chief Scientist of Baidu.', 'position': 2}, {'title': 'DeepLearning.AI - LinkedIn', 'link':         │
│  'https://www.linkedin.com/company/deeplearningai', 'snippet': 'DeepLearning.AI is making a world-class AI      │
│  education accessible to people around the globe. DeepLearning.AI was founded by Andrew Ng, a global leader in  │
│  AI.', 'position': 3}, {'title': 'DEEPLEARNING.AI LLC Company Profile | Palo Alto, CA', 'link':                 │
│  'https://www.dnb.com/business-directory/company-profiles.deeplearningai_llc.60c8a9eeade896ced9837d0fef6f19ec.  │
│  html', 'snippet': 'Find company research, competitor information, contact details & financial data for         │
│  DEEPLEARNING.AI LLC of Palo Alto, CA. Get the latest business insights ...', 'position': 4}, {'title':         │
│  'DeepLearning.AI: Start or Advance Your Career in AI', 'link': 'https://www.deeplearning.ai/', 'snippet':      │
│  'Join over 7 million people learning how to use and build AI. AI Courses and Specializations. Build a          │
│  foundation of machine learning and AI skills.', 'position': 5}, {'title': 'DeepLearning.AI - Overview, News &  │
│  Similar companies - ZoomInfo', 'link': 'https://www.zoominfo.com/c/deeplearningai-inc/462132321', 'snippet':   │
│  'DeepLearning.AI is an education technology company that is empowering the global workforce to build an        │
│  AI-powered future through world-class education.', 'position': 6}, {'title': 'Deeplearning - 2025 Co...        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sales Representative                                                                                    │
│                                                                                                                 │
│  Thought: Thought: The search results provide a good overview of DeepLearningAI. I can see that it's an         │
│  education technology company founded by Andrew Ng, focused on AI education. I'll now search for more specific  │
│  information about key personnel and recent milestones.                                                         │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  positive                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Thought: Thought: The sentiment analysis confirms a positive tone for the second email draft. Now, I'll draft  │
│  a third email focusing on community engagement.                                                                │
│                                                                                                                 │
│  Using Tool: Sentiment Analysis Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"argument\": \"Subject: Fostering a Thriving AI Learning Community - Collaboration with CrewAI\\n\\nDear A  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  positive                                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│  You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:          │
│                                                                                                                 │
│  Tool Name: Search the internet with Serper                                                                     │
│  Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the         │
│  internet', 'type': 'str'}}                                                                                     │
│  Tool Description: A tool that can be used to search the internet with a search_query. Supports different       │
│  search types: 'search' (default), 'news'                                                                       │
│  Tool Name: Sentiment Analysis Tool                                                                             │
│  Tool Arguments: {'argument': {'description': None, 'type': 'str'}}                                             │
│  Tool Description: Analyzes the sentiment of text to ensure positive and engaging communication.                │
│                                                                                                                 │
│  IMPORTANT: Use the following format in your response:                                                          │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, only one name of [Search the internet with Serper, Sentiment Analysis Tool], just  │
│  the name, exactly as it's written.                                                                             │
│  Action Input: the input to the action, just a simple JSON object, enclosed in curly braces, using " to wrap    │
│  keys and values.                                                                                               │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│                                                                                                                 │
│  Once all necessary information is gathered, return the following format:                                       │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now know the final answer                                                                           │
│  Final Answer: the final answer to the original input question                                                  │
│  ```                                                                                                            │
│                                                                                                                 │
╰───────────────────────────────────────────────────────

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  **Email Draft 1: Personalized Learning**                                                                       │
│                                                                                                                 │
│  Subject: Revolutionizing AI Education with Personalized Learning - A Proposal for DeepLearningAI               │
│                                                                                                                 │
│  Dear Andrew,                                                                                                   │
│                                                                                                                 │
│  I've long admired your commitment to democratizing AI education through DeepLearningAI. Your recent AI Dev 25  │
│  event, as highlighted by Jonathan Heyne, truly showcased the vibrant community you've cultivated.              │
│                                                                                                                 │
│  At CrewAI, we're equally passionate about leveraging AI to enhance learning experiences. We believe our        │
│  multi-agent systems can be seamlessly integrated into DeepLearningAI's courses to provide students with        │
│  personalized learning journeys. Imagine AI agents acting as virtual teaching assistants, offering customized   │
│  feedback, answering questions, and guiding students through complex concepts – all tailored to their           │
│  individual needs and learning styles.                                                                          │
│                                                                                                                 │
│  We've been particularly impressed with DeepLearningAI's focus on practical application and real-world skills.  │
│  Our AI agents can further support this by providing personalized career guidance, helping students identify    │
│  relevant job opportunities and develop their professional networks.                                            │
│                                                                                                                 │
│  I'd be thrilled to discuss how CrewAI can help DeepLearningAI further elevate its educational offerings and    │
│  empower even more individuals to succeed in the AI era. Would you be open to a brief call next week?           │
│                                                                                                                 │
│  Sincerely,                                                                                                     │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│  Lead Sales Representative                                                                                      │
│                                                                                                                 │
│  **Email Draft 2: Content Creation**                                                                            │
│                                                        

In [38]:
from IPython.display import Markdown
import re

content = result.raw
markdown_content = re.sub(r'```(?:markdown)?\n?(.*?)\n?```', r'\1', content, flags=re.DOTALL).strip()
Markdown(markdown_content)

**Email Draft 1: Personalized Learning**

Subject: Revolutionizing AI Education with Personalized Learning - A Proposal for DeepLearningAI

Dear Andrew,

I've long admired your commitment to democratizing AI education through DeepLearningAI. Your recent AI Dev 25 event, as highlighted by Jonathan Heyne, truly showcased the vibrant community you've cultivated.

At CrewAI, we're equally passionate about leveraging AI to enhance learning experiences. We believe our multi-agent systems can be seamlessly integrated into DeepLearningAI's courses to provide students with personalized learning journeys. Imagine AI agents acting as virtual teaching assistants, offering customized feedback, answering questions, and guiding students through complex concepts – all tailored to their individual needs and learning styles.

We've been particularly impressed with DeepLearningAI's focus on practical application and real-world skills. Our AI agents can further support this by providing personalized career guidance, helping students identify relevant job opportunities and develop their professional networks.

I'd be thrilled to discuss how CrewAI can help DeepLearningAI further elevate its educational offerings and empower even more individuals to succeed in the AI era. Would you be open to a brief call next week?

Sincerely,

[Your Name]
Lead Sales Representative

**Email Draft 2: Content Creation**

Subject: Supercharge Your Content Creation with AI - A Partnership Opportunity for DeepLearningAI

Dear Andrew,

Following the success of AI Dev 25, I wanted to reach out and explore how CrewAI could help DeepLearningAI scale its content creation efforts.

We understand the importance of high-quality educational materials in delivering effective AI training. Our AI-powered tools can assist DeepLearningAI in creating and curating engaging content more efficiently. Imagine AI agents automating tasks such as generating quizzes, summarizing research papers, and even translating course materials into multiple languages.

This would not only save your team valuable time but also ensure that your content remains fresh, relevant, and accessible to a global audience. We believe this aligns perfectly with DeepLearningAI's mission of making AI education accessible to everyone.

I'd love to schedule a brief demo to showcase our content creation capabilities and discuss how they can benefit DeepLearningAI. Are you available for a quick chat sometime next week?

Best regards,

[Your Name]
Lead Sales Representative

**Email Draft 3: Community Engagement**

Subject: Fostering a Thriving AI Learning Community - Collaboration with CrewAI

Dear Andrew,

I was very impressed by the community engagement at DeepLearningAI's recent AI Dev 25 event. It's clear you understand the power of community in fostering learning and innovation.

At CrewAI, we believe AI can play a crucial role in enhancing community engagement. Our AI agents can be used to foster a more interactive and supportive learning environment within DeepLearningAI's courses. Imagine AI agents moderating online forums, facilitating discussions, and connecting students with relevant resources and mentors.

This would not only create a more engaging learning experience but also help students build valuable connections and expand their professional networks. We believe this aligns perfectly with DeepLearningAI's commitment to empowering individuals and organizations with the skills they need to succeed in the AI era.

I'd be delighted to discuss how CrewAI can help DeepLearningAI further cultivate its learning community. Would you be open to a brief conversation next week?

Sincerely,

[Your Name]
Lead Sales Representative

In [36]:
from IPython.display import Markdown
import re

content = result.raw
markdown_content = re.sub(r'```(?:markdown)?\n?(.*?)\n?```', r'\1', content, flags=re.DOTALL).strip()
Markdown(markdown_content)

Email Draft 1:

Subject: Revolutionizing AI Education: How [Your Company] Can Empower DeepLearningAI's Next Chapter

Dear Andrew,

I've been consistently impressed by DeepLearningAI's commitment to democratizing AI education, especially with the recent launch of the Data Analytics Professional Certificate. Congratulations on this significant achievement!

At [Your Company], we share your passion for accessible, high-quality learning. We've been closely following DeepLearningAI's journey and are particularly excited about your strategic partnership with Bain & Company. We believe our platform can be a valuable asset in scaling your educational initiatives and enhancing the learning experience for your global audience.

Specifically, we can help DeepLearningAI:

*   **Scale content delivery:** Our robust platform ensures seamless delivery of your courses, even with rapidly growing user base.
*   **Personalize learning paths:** Our AI-powered engine tailors content to individual learner needs, boosting engagement and outcomes.
*   **Foster a thriving community:** Our platform facilitates collaboration and knowledge sharing among your learners.
*   **Integrate with enterprise systems:** Seamlessly integrate DeepLearningAI's content into Bain & Company's client training programs.
*   **Unlock data-driven insights:** Our analytics tools provide valuable data on learner behavior and course performance.

I'd love to schedule a brief call to discuss how [Your Company] can support DeepLearningAI in achieving its ambitious goals. Would you be open to a 15-minute conversation next week?

Thank you for your time and consideration.

Sincerely,

[Your Name]
Lead Sales Representative

Email Draft 2:

Subject: Enhancing DeepLearningAI's Impact Through Strategic Technology

Dear Andrew,

I was very interested to learn about DeepLearningAI's strategic partnership with Bain & Company. This is a fantastic step towards accelerating AI transformation for businesses worldwide.

At [Your Company], we specialize in providing scalable and innovative solutions for online learning platforms. We believe our platform can significantly enhance DeepLearningAI's ability to deliver impactful training programs, especially in the context of your collaboration with Bain & Company.

Here's how we can help:

*   **Seamless Enterprise Integration:** Facilitate the integration of DeepLearningAI's courses into Bain & Company's client training programs, ensuring a cohesive learning experience.
*   **Personalized Learning at Scale:** Leverage our AI-powered personalization engine to tailor learning paths for individual Bain & Company clients, maximizing knowledge retention and application.
*   **Data-Driven Optimization:** Provide comprehensive data analytics and reporting to track learner progress, measure program effectiveness, and identify areas for improvement.
*   **Scalable Content Delivery:** Ensure reliable and high-quality content delivery to a global audience, even with increasing demand.
*   **Community Engagement:** Foster a vibrant community of learners through discussion forums, peer-to-peer mentoring, and collaborative projects.

I'd be delighted to discuss how [Your Company] can contribute to the success of DeepLearningAI's partnership with Bain & Company. Would you be available for a brief introductory call?

Best regards,

[Your Name]
Lead Sales Representative

Email Draft 3:

Subject: Empowering Data Scientists: [Your Company]'s Support for DeepLearningAI's New Certificate

Dear Andrew,

Congratulations on the launch of DeepLearningAI's Data Analytics Professional Certificate! This is a significant step in equipping individuals with the skills needed to thrive in today's data-driven world.

At [Your Company], we're passionate about enabling effective and engaging online learning experiences. We believe our platform can be a powerful tool for DeepLearningAI in delivering this new certificate program and maximizing its impact.

Here's how we can support your efforts:

*   **Interactive Learning Environment:** Enhance the learning experience with interactive exercises, simulations, and real-world case studies.
*   **Personalized Feedback and Support:** Provide learners with personalized feedback and support through AI-powered tutoring and mentoring.
*   **Scalable Assessment Tools:** Streamline the assessment process with automated grading and feedback, allowing instructors to focus on student engagement.
*   **Data-Driven Insights:** Track learner progress and identify areas for improvement with comprehensive data analytics and reporting.
*   **Community Building:** Foster a collaborative learning environment through discussion forums, peer-to-peer support, and group projects.

I'd love to explore how [Your Company] can help DeepLearningAI make the Data Analytics Professional Certificate a resounding success. Would you be open to a brief conversation?

Sincerely,

[Your Name]
Lead Sales Representative